In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from collections import Counter
import xgboost as xgb

# 1. Load & initial clean
df = pd.read_csv("totalload_new.csv", engine="python")

# 1.1 Drop entirely empty columns
df = df.dropna(axis=1, how="all")

# 1.2 Parse and rename timestamp
df['ds'] = pd.to_datetime(df['BASE_DT'], format='%d-%m-%y %H:%M', errors='coerce')
df = df.dropna(subset=['ds']).sort_values('ds').reset_index(drop=True)

# 1.3 Impute numeric columns (including TOTAL_LOAD and weather)
numeric_cols = df.select_dtypes(include=['int64','float64']).columns
df[numeric_cols] = SimpleImputer(strategy="mean").fit_transform(df[numeric_cols])

# 1.4 Create target column for Prophet
df['y'] = df['TOTAL_LOAD']

# 2. Prophet decomposition
prophet_df = df[['ds','y']]
m = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)
m.fit(prophet_df)
forecast = m.predict(prophet_df)
df['trend']       = forecast['trend'].values
df['seasonality'] = forecast['yearly'].values

# 3. Feature engineering
df['hour']        = df['ds'].dt.hour
df['day_of_week'] = df['ds'].dt.dayofweek
df['is_weekend']  = (df['day_of_week'] >= 5).astype(int)

# 3.1 Load lags and rolling
df['lag_1']      = df['y'].shift(1)
df['lag_24']     = df['y'].shift(24)
df['roll3_mean'] = df['y'].rolling(3).mean()
df['roll3_std']  = df['y'].rolling(3).std()

# 3.2 Weather feature prefixes (customize these)
weather_prefixes = ['JEJU_','GOSAN_','SUNGSAN_','SEOGWIPO_']
weather_feats = [c for c in df.columns if any(c.startswith(p) for p in weather_prefixes)]

# 3.3 Create weather lags
for feat in weather_feats:
    df[f'{feat}_lag1']  = df[feat].shift(1)
    df[f'{feat}_lag24'] = df[feat].shift(24)

# 3.4 Drop rows with any NaNs from shifting/rolling
df = df.dropna().reset_index(drop=True)

# 4. Train/test split
cutoff = pd.to_datetime('2019-01-01')
train = df[df['ds'] < cutoff].copy()
test  = df[df['ds'] >= cutoff].copy()

# 5. Prepare feature lists
base_feats    = ['trend','seasonality','hour','day_of_week','is_weekend',
                 'lag_1','lag_24','roll3_mean','roll3_std']
lagged_weather = [c for c in df.columns if c.endswith('_lag1') or c.endswith('_lag24')]
full_feats     = base_feats + weather_feats + lagged_weather

# 6. Hybrid Embedded + Stability Feature Selection
# 6.1 Filter out constant features
vt = VarianceThreshold(threshold=1e-8)
vt.fit(train[full_feats])
filtered_feats = [f for f,keep in zip(full_feats, vt.get_support()) if keep]

# 6.2 Stability selection
B, K, tau = 30, 15, 0.6
counts = Counter()

for b in range(B):
    sample = train.sample(frac=1.0, replace=True, random_state=b)
    dtrain_b = xgb.DMatrix(sample[filtered_feats], label=sample['y'])
    model_b = xgb.train(
        {'objective':'reg:squarederror','tree_method':'hist',
         'max_depth':4,'learning_rate':0.1,'seed':b},
        dtrain_b, num_boost_round=50, verbose_eval=False
    )
    imp = model_b.get_score(importance_type='gain')
    topk = [f for f,_ in sorted(imp.items(), key=lambda x: x[1], reverse=True)[:K]]
    counts.update(topk)

selected_feats = [f for f,c in counts.items() if c >= tau * B]
print("Selected features:", selected_feats)

# 7. Cross-Validation & Final Model Training
dtrain = xgb.DMatrix(train[selected_feats], label=train['y'])
dtest  = xgb.DMatrix(test[selected_feats],  label=test ['y'])

params = {
    'objective':'reg:squarederror','tree_method':'hist',
    'learning_rate':0.05,'max_depth':6,
    'subsample':0.8,'colsample_bytree':0.8,
    'eval_metric':'mae','seed':42
}
cv = xgb.cv(
    params, dtrain, num_boost_round=1000,
    nfold=5, early_stopping_rounds=20,
    metrics='mae', as_pandas=True, seed=42
)
best_n = len(cv)

bst = xgb.train(params, dtrain, num_boost_round=best_n)

# 8. Predict & Evaluate
y_pred = bst.predict(dtest)
y_true = test['y'].values


mae  = mean_absolute_error(y_true, y_pred)
mse  = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_true, y_pred) * 100

print(f"Test MAE:  {mae:.2f}")
print(f"Test MSE:  {mse:.2f}")
print(f"Test RMSE: {rmse:.2f}")
print(f"Test MAPE: {mape:.2f}%")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# 1. Build results DataFrame
df_res = test[['ds','y']].copy()
df_res['pred'] = y_pred
df_res['Day']   = df_res['ds'].dt.day_name()
df_res['Week']  = df_res['ds'].dt.isocalendar().week
df_res['Month'] = df_res['ds'].dt.month_name()

# 2. Metric computation function
def compute_metrics(g):
    y_true, y_pr = g['y'], g['pred']
    mae  = mean_absolute_error(y_true, y_pr)
    mse  = mean_squared_error(y_true, y_pr)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_true, y_pr)*100
    return pd.Series({'MAE':mae,'MSE':mse,'RMSE':rmse,'MAPE (%)':mape})

# 3. Compute tables
daily   = df_res.groupby('Day').apply(compute_metrics).reindex(
            ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

monthly = df_res.groupby('Month').apply(compute_metrics).reindex([
            'January','February','March','April','May','June',
            'July','August','September','October','November','December'])

# 4. Display
print("### Daily Metrics")
print(daily.to_markdown())

print("\n### Monthly Metrics")
print(monthly.to_markdown())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sns.set(style="whitegrid", font_scale=1.0)

# Recompute gain_full from your trained booster
gain = bst.get_score(importance_type='gain')
gain_full = {f: gain.get(f, 0.0) for f in selected_feats}

# Prepare data
rounds    = np.arange(len(cv))
train_mu  = cv['train-mae-mean']
valid_mu  = cv['test-mae-mean']
residuals = y_true - y_pred

# Top-10 selected features by gain
fi = pd.Series(gain_full).sort_values().tail(10)

# Create 2×2 subplot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
(ax1, ax2), (ax3, ax4) = axes

# A) Learning Curve
ax1.plot(rounds, train_mu, label='Train MAE', lw=2)
ax1.plot(rounds, valid_mu, label='Val MAE',   lw=2)
ax1.axvline(best_n, color='gray', ls='--', label=f'Optimal ({best_n})')
ax1.set(xlabel='Boosting Round', ylabel='MAE', title='Learning Curve')
ax1.legend()
ax1.text(-0.05, 1.05, 'A', transform=ax1.transAxes,
         fontsize=16, fontweight='bold')

# B) Actual vs. Predicted
ax2.plot(test['ds'], y_true, label='Actual',   color='#2e7d32', lw=1)
ax2.plot(test['ds'], y_pred, label='Predicted', color='#e64a19', lw=1)
ax2.set(xlabel='Date', ylabel='Load', title='Actual vs. Predicted')
ax2.legend()
ax2.text(-0.05, 1.05, 'B', transform=ax2.transAxes,
         fontsize=16, fontweight='bold')

# C) Residual Distribution
sns.histplot(residuals, kde=True, stat="density", ax=ax3, color='#9ccc65')
ax3.set(xlabel='Residual', title='Residual Distribution')
ax3.text(-0.05, 1.05, 'C', transform=ax3.transAxes,
         fontsize=16, fontweight='bold')

# D) Top-10 Feature Importances
features = fi.index.tolist()
importances = fi.values
cmap = plt.get_cmap('tab10')
colors = [cmap(i % 10) for i in range(len(features))]
ax4.barh(features, importances, color=colors)
ax4.set(xlabel='Gain Importance', title='Top-10 Feature Importances')
ax4.text(-0.05, 1.05, 'D', transform=ax4.transAxes,
         fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
print(f"Total features before selection: {len(full_feats)}")
print(f"Features selected for training: {len(selected_feats)}")


import matplotlib.pyplot as plt

n_full     = len(full_feats)
n_selected = len(selected_feats)

plt.figure(figsize=(5,4))
bars = plt.bar(
    ['All features','Selected'],
    [n_full, n_selected],
    color=['#e28743','#1f77b4']
)
plt.ylabel('Feature Count')
plt.title('Features Before vs. After Selection')
for bar in bars:
    h = bar.get_height()
    plt.text(bar.get_x()+bar.get_width()/2, h+1, str(h),
             ha='center', va='bottom', fontweight='bold')
plt.ylim(0, n_full*1.1)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 1. Compute counts
n_selected   = len(selected_feats)
n_unselected = len(full_feats) - n_selected
n_total      = len(full_feats)
pct_selected   = n_selected   / n_total * 100
pct_unselected = n_unselected / n_total * 100

# 2. Set up figure with two subplots: horizontal bar + donut
fig, (ax_bar, ax_donut) = plt.subplots(
    1, 2, figsize=(14, 5),
    gridspec_kw={'width_ratios': [2, 1]}
)

# 3. Horizontal Bar Chart (counts only)
labels = ['Total Features', 'Unselected', 'Selected']
counts = [n_total, n_unselected, n_selected]
colors = ['#ff6f00', '#1f77b4', '#9ccc65']

y_pos = range(len(labels))
ax_bar.barh(y_pos, counts, color=colors, edgecolor='none')
ax_bar.set_yticks(y_pos)
ax_bar.set_yticklabels(labels, fontsize=12)
ax_bar.invert_yaxis()  # highest at top
ax_bar.set_xlabel('Count', fontsize=12)
ax_bar.set_title('Feature Counts', fontsize=14, pad=10)
ax_bar.grid(axis='x', linestyle='--', alpha=0.5)

# Annotate counts (no percentages)
for i, count in enumerate(counts):
    ax_bar.text(
        count + n_total * 0.01, i,
        str(count),
        va='center', fontsize=11, fontweight='bold'
    )

ax_bar.set_xlim(0, n_total * 1.1)

# 4. Donut Chart for Selected vs Unselected
sizes = [n_selected, n_unselected]
d_colors = ['#9ccc65', '#9e9e9e']
wedges, texts = ax_donut.pie(
    sizes, labels=None, colors=d_colors,
    startangle=90, wedgeprops={'width': 0.5, 'edgecolor': 'white'}
)
ax_donut.set_title('Selection Proportion', fontsize=14, pad=10)

# Legend with percentages
ax_donut.legend(
    wedges,
    [f'Selected ({pct_selected:.1f}%)', f'Unselected ({pct_unselected:.1f}%)'],
    title='',
    loc='center',
    bbox_to_anchor=(0.5, -0.1),
    ncol=1,
    fontsize=12
)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 9. Plot Feature Importance with Different Colors
# Get feature importance scores from the final model
importance = bst.get_score(importance_type='gain')
importance = {k: v for k, v in importance.items() if k in selected_feats}

# Sort features by importance
sorted_importance = dict(sorted(importance.items(), key=lambda x: x[1], reverse=True))
features = list(sorted_importance.keys())
scores = list(sorted_importance.values())

# Generate a colormap for different colors
colors = plt.cm.tab20(np.linspace(0, 1, len(features)))

# Create bar plot with different colors for each bar
plt.figure(figsize=(10, 6))
plt.barh(features, scores, color=colors)
plt.xlabel('Feature Importance (Gain)')
plt.title('Feature Importance for Selected Features')
plt.gca().invert_yaxis()  # Highest importance at the top
plt.tight_layout()

# Save and show the plot
plt.savefig('feature_importance.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.inspection import partial_dependence
from xgboost import XGBRegressor, DMatrix
from collections import defaultdict

# 11. Overlaid Partial Dependence and Residual Plots for Common Weather Variables

# Filter weather features that are in selected_feats
weather_in_sel = [f for f in weather_feats if f in selected_feats]

# Group weather features by common variable (e.g., DI, TEMP)
# Assume feature format is {location}_{variable}, e.g., JEJU_DI
weather_groups = defaultdict(list)
prefixes = ['JEJU_', 'GOSAN_', 'SUNGSAN_', 'SEOGWIPO_']
for feat in weather_in_sel:
    for prefix in prefixes:
        if feat.startswith(prefix):
            variable = feat[len(prefix):]  # Extract variable (e.g., DI)
            weather_groups[variable].append(feat)
            break

# Prepare test data for residual calculations
df_plot = test.copy()
df_plot['y_pred'] = bst.predict(DMatrix(test[selected_feats]))
df_plot['residual'] = df_plot['y'] - df_plot['y_pred']

# Create an XGBRegressor model equivalent to bst for partial dependence
params = {
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'learning_rate': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'mae',
    'seed': 42
}
xgb_regressor = XGBRegressor(**params)
xgb_regressor.fit(train[selected_feats], train['y'])

# Create a single figure with subplots for each common variable
n_variables = len(weather_groups)
if n_variables > 0:
    fig, axes = plt.subplots(n_variables, 2, figsize=(10, n_variables * 3), squeeze=False)

    # Use tab10 colormap for distinct colors per location
    colors = plt.cm.tab10(np.linspace(0, 1, len(prefixes)))
    location_colors = {prefix: colors[i] for i, prefix in enumerate(prefixes)}

    for i, (variable, features) in enumerate(weather_groups.items()):
        # 11.1 Partial Dependence Plot (Left Column)
        for feat in features:
            feature_idx = selected_feats.index(feat)
            pd_results = partial_dependence(
                xgb_regressor,
                X=train[selected_feats],
                features=[feature_idx],
                grid_resolution=50,
                kind='average'
            )
            # Extract location prefix for labeling and coloring
            location = next(prefix for prefix in prefixes if feat.startswith(prefix))
            axes[i, 0].plot(
                pd_results["grid_values"][0],
                pd_results["average"][0],
                label=location[:-1],  # Remove underscore for label
                color=location_colors.get(location, 'black')
            )
        axes[i, 0].set_xlabel(variable)
        axes[i, 0].set_ylabel('Partial Dependence')
        axes[i, 0].set_title(f'Partial Dependence: {variable}')
        axes[i, 0].legend()

        # 11.2 Residual vs. Feature Scatter Plot (Right Column)
        for feat in features:
            location = next(prefix for prefix in prefixes if feat.startswith(prefix))
            sns.scatterplot(
                x=feat,
                y='residual',
                data=df_plot,
                alpha=0.4,
                s=10,
                label=location[:-1],
                color=location_colors.get(location, 'black'),
                ax=axes[i, 1]
            )
        axes[i, 1].axhline(0, color='black', linestyle='--', linewidth=1)
        axes[i, 1].set_xlabel(variable)
        axes[i, 1].set_ylabel('Residual (Actual - Predicted)')
        axes[i, 1].set_title(f'Residual vs. {variable}')
        axes[i, 1].legend()

    plt.tight_layout()
    plt.savefig('overlaid_weather_plots.png')
    plt.show()
else:
    print("No weather features found in selected_feats. Skipping overlaid plot.")

## Model Comparison

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import time
import pickle

import pandas as pd
import numpy as np
from prophet import Prophet

from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error,
    mean_absolute_percentage_error, r2_score
)

import xgboost as xgb
from collections import Counter

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import optuna
from optuna.samplers import TPESampler

from statsmodels.tsa.arima.model import ARIMA

import matplotlib.pyplot as plt


##############################################################################################################
# GPU / DEVICE SETUP
##############################################################################################################
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("\n✓ GPU Detected: Using CUDA")
    print("CUDA Version:", torch.version.cuda)
else:
    device = torch.device("cpu")
    print("\n× GPU NOT detected — using CPU")

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True


##############################################################################################################
# UTILITIES FOR COMPLEXITY
##############################################################################################################
def get_model_size_mb(model, framework="torch"):
    """
    Save model to a temporary file and measure size in MB.
    framework: "torch", "xgb", "pickle"
    """
    if framework == "torch":
        tmp_file = "tmp_model.pth"
        torch.save(model.state_dict(), tmp_file)
    elif framework == "xgb":
        tmp_file = "tmp_model.json"
        model.save_model(tmp_file)
    else:  # generic pickle (Prophet, sklearn, statsmodels, etc.)
        tmp_file = "tmp_model.pkl"
        with open(tmp_file, "wb") as f:
            pickle.dump(model, f)

    size_mb = os.path.getsize(tmp_file) / (1024 * 1024)
    os.remove(tmp_file)
    return size_mb


def measure_inference_time(model, X, framework="torch"):
    """
    Measure total and per-sample inference time.
    Avoid tensor.numpy() to work around no-NumPy PyTorch builds.
    """
    if framework == "torch":
        model.eval()
        t0 = time.time()
        with torch.no_grad():
            _ = model(torch.tensor(X).float().to(device)).cpu()  # no .numpy()
        t1 = time.time()
    elif framework == "xgb":
        t0 = time.time()
        _ = model.predict(xgb.DMatrix(X.astype("float32")))
        t1 = time.time()
    elif framework == "sk":  # sklearn-like
        t0 = time.time()
        _ = model.predict(X)
        t1 = time.time()
    elif framework == "prophet":
        t0 = time.time()
        _ = model.predict(X)
        t1 = time.time()
    else:
        t0 = time.time()
        _ = model.predict(X)
        t1 = time.time()

    total = t1 - t0
    per_sample = total / len(X)
    return total, per_sample


def gpu_memory_used_mb():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        used = torch.cuda.max_memory_allocated() / (1024 * 1024)
        torch.cuda.reset_peak_memory_stats()
        return used
    else:
        return 0.0


##############################################################################################################
# STEP 1: Load + Preprocess
##############################################################################################################
df = pd.read_csv("totalload_new.csv", engine="python")
df = df.dropna(axis=1, how="all")

df["ds"] = pd.to_datetime(df["BASE_DT"], format="%d-%m-%y %H:%M", errors="coerce")
df = df.dropna(subset=["ds"]).sort_values("ds").reset_index(drop=True)

numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns
df[numeric_cols] = SimpleImputer(strategy="mean").fit_transform(df[numeric_cols])

df["y"] = df["TOTAL_LOAD"]


##############################################################################################################
# STEP 2: Prophet Decomposition (for features)
##############################################################################################################
m_decomp = Prophet(weekly_seasonality=True, yearly_seasonality=True, daily_seasonality=True)
m_decomp.fit(df[["ds", "y"]])
fc = m_decomp.predict(df[["ds"]])

df["trend"] = fc["trend"].values
df["seasonality"] = fc["yearly"].values


##############################################################################################################
# STEP 3: Feature Engineering
##############################################################################################################
df["hour"] = df["ds"].dt.hour
df["dow"] = df["ds"].dt.dayofweek
df["weekend"] = (df["dow"] >= 5).astype(int)

df["lag1"] = df["y"].shift(1)
df["lag24"] = df["y"].shift(24)
df["roll3_mean"] = df["y"].rolling(3).mean()
df["roll3_std"] = df["y"].rolling(3).std()

weather_prefix = ["JEJU_", "GOSAN_", "SUNGSAN_", "SEOGWIPO_"]
weather_feats = [c for c in df.columns if any(c.startswith(p) for p in weather_prefix)]

for wf in weather_feats:
    df[f"{wf}_lag1"] = df[wf].shift(1)
    df[f"{wf}_lag24"] = df[wf].shift(24)

df = df.dropna().reset_index(drop=True)

base_feats = ["trend","seasonality","hour","dow","weekend",
              "lag1","lag24","roll3_mean","roll3_std"]
lag_weather = [c for c in df.columns if c.endswith("_lag1") or c.endswith("_lag24")]
full_feats = base_feats + weather_feats + lag_weather


##############################################################################################################
# STEP 4: Train/Test Split
##############################################################################################################
cut = pd.to_datetime("2019-01-01")
train = df[df["ds"] < cut].copy()
test  = df[df["ds"] >= cut].copy()


##############################################################################################################
# STEP 5: Feature Selection (Variance + Stability) using CPU XGBoost (hist)
##############################################################################################################
vt = VarianceThreshold(1e-8)
vt.fit(train[full_feats])
filtered = [f for f, keep in zip(full_feats, vt.get_support()) if keep]

B, K, tau = 30, 15, 0.6
counts = Counter()

for b in range(B):
    samp = train.sample(frac=1.0, replace=True, random_state=b)
    dmat = xgb.DMatrix(samp[filtered].astype(np.float32), label=samp["y"].astype(np.float32))
    model_b = xgb.train(
        {"objective": "reg:squarederror", "tree_method": "hist"},
        dmat, num_boost_round=50
    )
    imp = model_b.get_score(importance_type="gain")
    topk = sorted(imp.items(), key=lambda x: x[1], reverse=True)[:K]
    counts.update([f for f,_ in topk])

selected_feats = [f for f, c in counts.items() if c >= B * tau]
#print(f"\nSelected Features: {len(selected_feats)}")


##############################################################################################################
# STEP 6: Scaling + Sequence Preparation
##############################################################################################################
scX = StandardScaler()
scY = StandardScaler()

Xtr = scX.fit_transform(train[selected_feats])
Xts = scX.transform(test[selected_feats])

ytr = scY.fit_transform(train["y"].values.reshape(-1,1)).ravel()
yts = scY.transform(test["y"].values.reshape(-1,1)).ravel()


def create_sequences(X, y, L):
    Xs, ys = [], []
    for i in range(len(X)-L):
        Xs.append(X[i:i+L])
        ys.append(y[i+L])
    return np.array(Xs), np.array(ys)


LOOKBACK = 24
Xtr_seq, ytr_seq = create_sequences(Xtr, ytr, LOOKBACK)
Xts_seq, yts_seq = create_sequences(Xts, yts, LOOKBACK)


##############################################################################################################
# MODEL 1 — ProphetBoost (XGBoost, CPU hist)
##############################################################################################################
print("\n=== ProphetBoost (XGBoost, hist) ===")
dtrain = xgb.DMatrix(Xtr.astype("float32"), label=ytr.astype("float32"))
dtest  = xgb.DMatrix(Xts.astype("float32"), label=yts.astype("float32"))

xgb_params = {
    "objective": "reg:squarederror",
    "tree_method": "hist",   # CPU histogram
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "eval_metric": "mae"
}

cv = xgb.cv(xgb_params, dtrain, 1000, nfold=5, early_stopping_rounds=20,
            metrics="mae", as_pandas=True)
best_rounds = len(cv)

t0 = time.time()
xgb_model = xgb.train(xgb_params, dtrain, best_rounds)
train_time_xgb = time.time() - t0

infer_total_xgb, infer_per_xgb = measure_inference_time(
    xgb_model, Xts, framework="xgb"
)
size_xgb = get_model_size_mb(xgb_model, framework="xgb")
gpu_mem_xgb = gpu_memory_used_mb()

pred_xgb_scaled = xgb_model.predict(dtest)
pred_xgb = scY.inverse_transform(pred_xgb_scaled.reshape(-1,1)).ravel()


##############################################################################################################
# MODEL 2 — Transformer (VALID d_model % heads == 0)
##############################################################################################################
class TransformerModel(nn.Module):
    def __init__(self, d_model, num_heads, num_layers, dim_ff, input_dim):
        super().__init__()
        self.pe = self._pe(LOOKBACK, d_model)
        self.proj = nn.Linear(input_dim, d_model)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=num_heads,
            dim_feedforward=dim_ff, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, 1)

    def _pe(self, L, D):
        pos = torch.arange(L).unsqueeze(1)
        div = torch.exp(torch.arange(0, D, 2) * -(np.log(10000.0) / D))
        pe = torch.zeros(1, L, D)
        pe[0,:,0::2] = torch.sin(pos * div)
        pe[0,:,1::2] = torch.cos(pos * div)
        return pe

    def forward(self, x):
        x = self.proj(x) + self.pe[:, :x.size(1), :].to(device)
        x = self.encoder(x)
        return self.fc(x[:,-1,:])


def objective_tf(trial):

    d_model = trial.suggest_categorical("d_model", [32, 64, 128])
    num_heads = trial.suggest_categorical("num_heads", [1, 2, 4, 8])
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dim_ff = trial.suggest_categorical("dim_ff", [64,128,256])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch = trial.suggest_categorical("batch", [16,32,64])

    if d_model % num_heads != 0:
        return float("inf")

    model = TransformerModel(d_model, num_heads, num_layers, dim_ff,
                             input_dim=Xtr_seq.shape[2]).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    loader = DataLoader(TensorDataset(
        torch.tensor(Xtr_seq).float(),
        torch.tensor(ytr_seq).float().unsqueeze(-1)
    ), batch_size=batch, shuffle=False)

    model.train()
    for _ in range(20):
        for xb,yb in loader:
            xb,yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward()
            opt.step()

    model.eval()
    with torch.no_grad():
        pred_t = model(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
        pred = np.array(pred_t.tolist())
        loss = mean_squared_error(yts_seq[:len(pred)], pred)

    return loss


print("\nSearching Transformer HPO...")
study_tf = optuna.create_study(direction="minimize")
study_tf.optimize(objective_tf, n_trials=10)
best_tf = study_tf.best_params
print("Transformer Best Params:", best_tf)

print("\n=== Transformer Model ===")
model_tf = TransformerModel(
    d_model=best_tf["d_model"],
    num_heads=best_tf["num_heads"],
    num_layers=best_tf["num_layers"],
    dim_ff=best_tf["dim_ff"],
    input_dim=Xtr_seq.shape[2]
).to(device)

opt_tf = torch.optim.Adam(model_tf.parameters(), lr=best_tf["lr"])
loss_fn = nn.MSELoss()

loader_tf = DataLoader(TensorDataset(
    torch.tensor(Xtr_seq).float(),
    torch.tensor(ytr_seq).float().unsqueeze(-1)
), batch_size=best_tf["batch"], shuffle=False)

t0 = time.time()
model_tf.train()
for _ in range(50):
    for xb,yb in loader_tf:
        xb,yb = xb.to(device), yb.to(device)
        opt_tf.zero_grad()
        loss = loss_fn(model_tf(xb), yb)
        loss.backward()
        opt_tf.step()
train_time_tf = time.time() - t0

infer_total_tf, infer_per_tf = measure_inference_time(
    model_tf, Xts_seq, framework="torch"
)
size_tf = get_model_size_mb(model_tf, framework="torch")
gpu_mem_tf = gpu_memory_used_mb()

model_tf.eval()
with torch.no_grad():
    pred_tf_t = model_tf(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
    pred_tf_scaled = np.array(pred_tf_t.tolist())
pred_tf = scY.inverse_transform(pred_tf_scaled.reshape(-1,1)).ravel()


##############################################################################################################
# MODEL 3 — DLINEAR
##############################################################################################################
class DLinear(nn.Module):
    def __init__(self, seq_len):
        super().__init__()
        self.fc = nn.Linear(seq_len, 1)

    def forward(self, x):
        x = x[:,:,0]
        return self.fc(x)


def objective_dl(trial):

    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch = trial.suggest_categorical("batch", [16,32,64])

    model = DLinear(LOOKBACK).to(device)
    opti = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    loader = DataLoader(TensorDataset(
        torch.tensor(Xtr_seq).float(),
        torch.tensor(ytr_seq).float().unsqueeze(-1)
    ), batch_size=batch, shuffle=False)

    model.train()
    for _ in range(20):
        for xb,yb in loader:
            xb,yb = xb.to(device), yb.to(device)
            opti.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward()
            opti.step()

    model.eval()
    with torch.no_grad():
        pred_t = model(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
        pred = np.array(pred_t.tolist())
        loss = mean_squared_error(yts_seq[:len(pred)], pred)

    return loss


print("\nSearching DLinear HPO...")
study_dl = optuna.create_study(direction="minimize")
study_dl.optimize(objective_dl, n_trials=10)
best_dl = study_dl.best_params
print("DLinear Best Params:", best_dl)

print("\n=== DLinear ===")
model_dl = DLinear(LOOKBACK).to(device)
opt_dl = torch.optim.Adam(model_dl.parameters(), lr=best_dl["lr"])
loss_fn = nn.MSELoss()

loader_dl = DataLoader(TensorDataset(
    torch.tensor(Xtr_seq).float(),
    torch.tensor(ytr_seq).float().unsqueeze(-1)
), batch_size=best_dl["batch"], shuffle=False)

t0 = time.time()
model_dl.train()
for _ in range(50):
    for xb,yb in loader_dl:
        xb,yb = xb.to(device), yb.to(device)
        opt_dl.zero_grad()
        loss = loss_fn(model_dl(xb), yb)
        loss.backward()
        opt_dl.step()
train_time_dl = time.time() - t0

infer_total_dl, infer_per_dl = measure_inference_time(
    model_dl, Xts_seq, framework="torch"
)
size_dl = get_model_size_mb(model_dl, framework="torch")
gpu_mem_dl = gpu_memory_used_mb()

model_dl.eval()
with torch.no_grad():
    pred_dl_t = model_dl(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
    pred_dl_scaled = np.array(pred_dl_t.tolist())
pred_dl = scY.inverse_transform(pred_dl_scaled.reshape(-1,1)).ravel()


##############################################################################################################
# MODEL 4 — DEEP RVFL ENSEMBLE
##############################################################################################################
print("\n=== Deep RVFL Ensemble ===")

class DeepRVFL(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim,256), nn.ReLU(),
            nn.Linear(256,256), nn.ReLU(),
            nn.Linear(256,1)
        )

    def forward(self,x):
        x = x.reshape(x.size(0),-1)
        return self.net(x)

rvfl_models = []
for _ in range(5):
    mdl = DeepRVFL(Xtr_seq.shape[1] * Xtr_seq.shape[2]).to(device)
    opti = torch.optim.Adam(mdl.parameters(), lr=1e-3)
    rvfl_models.append((mdl, opti))

loader_rvfl = DataLoader(TensorDataset(
    torch.tensor(Xtr_seq).float(),
    torch.tensor(ytr_seq).float().unsqueeze(-1)
), batch_size=32, shuffle=False)

t0 = time.time()
for mdl, opti in rvfl_models:
    mdl.train()
    for _ in range(20):
        for xb,yb in loader_rvfl:
            xb,yb = xb.to(device), yb.to(device)
            opti.zero_grad()
            loss = loss_fn(mdl(xb), yb)
            loss.backward()
            opti.step()
train_time_rvfl = time.time() - t0

t0 = time.time()
preds_rvfl_scaled = []
for mdl,_ in rvfl_models:
    mdl.eval()
    with torch.no_grad():
        p_t = mdl(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
        p = np.array(p_t.tolist())
        preds_rvfl_scaled.append(p)
infer_total_rvfl = time.time() - t0
infer_per_rvfl = infer_total_rvfl / len(Xts_seq)

pred_rvfl_scaled = np.mean(preds_rvfl_scaled, axis=0)
pred_rvfl = scY.inverse_transform(pred_rvfl_scaled.reshape(-1,1)).ravel()

size_rvfl = get_model_size_mb(rvfl_models[0][0], framework="torch")
gpu_mem_rvfl = gpu_memory_used_mb()


##############################################################################################################
# MODEL 5 — CNN-LSTM
##############################################################################################################
print("\nTraining CNN-LSTM...")

class CNNLSTM(nn.Module):
    """
    Conv1d over features (channels) + LSTM over time.
    """
    def __init__(self, input_dim, conv_channels=64, hidden_dim=128, num_layers=1):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim,
                               out_channels=conv_channels,
                               kernel_size=3,
                               padding=1)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(conv_channels, hidden_dim,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.relu(self.conv1(x))      # (B, C, T)
        x = x.permute(0, 2, 1)            # (B, T, C)
        out, _ = self.lstm(x)             # (B, T, H)
        return self.fc(out[:, -1, :])

model_cnnlstm = CNNLSTM(input_dim=Xtr_seq.shape[2]).to(device)
opt_cnnlstm = torch.optim.Adam(model_cnnlstm.parameters(), lr=1e-3)

loader_cnnlstm = DataLoader(
    TensorDataset(
        torch.tensor(Xtr_seq).float(),
        torch.tensor(ytr_seq).float().unsqueeze(-1)
    ),
    batch_size=64, shuffle=False
)

t0 = time.time()
model_cnnlstm.train()
for _ in range(50):
    for xb,yb in loader_cnnlstm:
        xb,yb = xb.to(device), yb.to(device)
        opt_cnnlstm.zero_grad()
        loss = loss_fn(model_cnnlstm(xb), yb)
        loss.backward()
        opt_cnnlstm.step()
train_time_cnnlstm = time.time() - t0

infer_total_cnnlstm, infer_per_cnnlstm = measure_inference_time(
    model_cnnlstm, Xts_seq, framework="torch"
)
size_cnnlstm = get_model_size_mb(model_cnnlstm, framework="torch")
gpu_mem_cnnlstm = gpu_memory_used_mb()

model_cnnlstm.eval()
with torch.no_grad():
    pred_cnnlstm_t = model_cnnlstm(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
    pred_cnnlstm_scaled = np.array(pred_cnnlstm_t.tolist())
pred_cnnlstm = scY.inverse_transform(pred_cnnlstm_scaled.reshape(-1, 1)).ravel()


##############################################################################################################
# MODEL 6 — CNN-ANN
##############################################################################################################
print("\nTraining CNN-ANN...")

class CNNANN(nn.Module):
    """
    Conv1d over features + global pooling + MLP.
    """
    def __init__(self, input_dim, conv_channels=64):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim,
                               out_channels=conv_channels,
                               kernel_size=3,
                               padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.mlp = nn.Sequential(
            nn.Linear(conv_channels, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)           # (B, F, T)
        x = self.relu(self.conv1(x))     # (B, C, T)
        x = self.pool(x).squeeze(-1)     # (B, C)
        return self.mlp(x)               # (B, 1)

model_cnnann = CNNANN(input_dim=Xtr_seq.shape[2]).to(device)
opt_cnnann = torch.optim.Adam(model_cnnann.parameters(), lr=1e-3)

loader_cnnann = DataLoader(
    TensorDataset(
        torch.tensor(Xtr_seq).float(),
        torch.tensor(ytr_seq).float().unsqueeze(-1)
    ),
    batch_size=64, shuffle=False
)

t0 = time.time()
model_cnnann.train()
for _ in range(50):
    for xb,yb in loader_cnnann:
        xb,yb = xb.to(device), yb.to(device)
        opt_cnnann.zero_grad()
        loss = loss_fn(model_cnnann(xb), yb)
        loss.backward()
        opt_cnnann.step()
train_time_cnnann = time.time() - t0

infer_total_cnnann, infer_per_cnnann = measure_inference_time(
    model_cnnann, Xts_seq, framework="torch"
)
size_cnnann = get_model_size_mb(model_cnnann, framework="torch")
gpu_mem_cnnann = gpu_memory_used_mb()

model_cnnann.eval()
with torch.no_grad():
    pred_cnnann_t = model_cnnann(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
    pred_cnnann_scaled = np.array(pred_cnnann_t.tolist())
pred_cnnann = scY.inverse_transform(pred_cnnann_scaled.reshape(-1, 1)).ravel()


##############################################################################################################
# MODEL 7 — ARIMA-LSTM (ARIMA + residual LSTM)
##############################################################################################################
print("\nTraining ARIMA-LSTM (ARIMA + residual LSTM)...")

t0 = time.time()
y_train_raw = train["y"].values
y_test_raw  = test["y"].values

arima_order = (5, 1, 0)
arima_model = ARIMA(y_train_raw, order=arima_order)
arima_result = arima_model.fit()

arima_pred_train = arima_result.predict(start=0, end=len(y_train_raw)-1)
arima_pred_test  = arima_result.predict(start=len(y_train_raw),
                                        end=len(y_train_raw)+len(y_test_raw)-1)

residual_train = y_train_raw - arima_pred_train
scRes = StandardScaler()
res_tr_scaled = scRes.fit_transform(residual_train.reshape(-1, 1)).ravel()

Xtr_res_seq, ytr_res_seq = create_sequences(Xtr, res_tr_scaled, LOOKBACK)

class ResidualLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

model_res_lstm = ResidualLSTM(input_dim=Xtr_res_seq.shape[2]).to(device)
opt_res = torch.optim.Adam(model_res_lstm.parameters(), lr=1e-3)

loader_res = DataLoader(
    TensorDataset(
        torch.tensor(Xtr_res_seq).float(),
        torch.tensor(ytr_res_seq).float().unsqueeze(-1)
    ),
    batch_size=64, shuffle=False
)

model_res_lstm.train()
for _ in range(50):
    for xb, yb in loader_res:
        xb, yb = xb.to(device), yb.to(device)
        opt_res.zero_grad()
        loss = loss_fn(model_res_lstm(xb), yb)
        loss.backward()
        opt_res.step()
train_time_arima_lstm = time.time() - t0

# Inference: ARIMA test + LSTM residual
t0 = time.time()
arima_test_seq = arima_pred_test[LOOKBACK:]
with torch.no_grad():
    res_pred_t = model_res_lstm(
        torch.tensor(Xts_seq).float().to(device)
    ).cpu().detach().view(-1)
    res_pred = np.array(res_pred_t.tolist())
pred_arima_lstm = arima_test_seq + res_pred
infer_total_arima_lstm = time.time() - t0
infer_per_arima_lstm = infer_total_arima_lstm / len(Xts_seq)

size_arima = get_model_size_mb(arima_result, framework="pickle")
size_res_lstm = get_model_size_mb(model_res_lstm, framework="torch")
size_arima_lstm = size_arima + size_res_lstm
gpu_mem_arima_lstm = gpu_memory_used_mb()


##############################################################################################################
# FINAL METRICS 
##############################################################################################################
def metrics(ytrue, ypred):
    mae = mean_absolute_error(ytrue, ypred)
    mse = mean_squared_error(ytrue, ypred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(ytrue, ypred)*100
    r2 = r2_score(ytrue, ypred)
    return mae, mse, rmse, mape, r2

# Align all sequence-based models to same horizon (RVFL length)
y_true_final = test["y"].values[-len(pred_rvfl):]

pb_pred         = pred_xgb[-len(y_true_final):]
tf_pred         = pred_tf[-len(y_true_final):]
dl_pred         = pred_dl[-len(y_true_final):]
rvfl_pred       = pred_rvfl
cnnlstm_pred    = pred_cnnlstm[-len(y_true_final):]
cnnann_pred     = pred_cnnann[-len(y_true_final):]
arima_lstm_pred = pred_arima_lstm[-len(y_true_final):]

In [ ]:
print("\n================== FINAL RESULTS ==================")
print("Model              MAE        MSE        RMSE       MAPE      R2")
print("--------------------------------------------------------------------------")
print(f"ProphetBoost   {metrics(y_true_final, pb_pred)}")
print(f"Transformer    {metrics(y_true_final, tf_pred)}")
print(f"DLinear        {metrics(y_true_final, dl_pred)}")
print(f"Deep RVFL      {metrics(y_true_final, rvfl_pred)}")
print(f"CNN-LSTM       {metrics(y_true_final, cnnlstm_pred)}")
print(f"CNN-ANN        {metrics(y_true_final, cnnann_pred)}")
print(f"ARIMA-LSTM     {metrics(y_true_final, arima_lstm_pred)}")
print("==========================================================================\n")


##############################################################################################################
# MODEL COMPLEXITY SUMMARY
##############################################################################################################
print("\n================ MODEL COMPLEXITY ==================")
print("Model          | Train(s) | Infer(s) | Infer/sample(ms) | Size(MB) | GPU Mem(MB)")
print("---------------------------------------------------------------------------------")
print(f"ProphetBoost   | {train_time_xgb:8.3f}     | {infer_total_xgb:8.3f}     | {infer_per_xgb*1000:12.4f}     | {size_xgb:8.2f}     | {gpu_mem_xgb:10.2f}")
print(f"Transformer    | {train_time_tf:8.3f}      | {infer_total_tf:8.3f}      | {infer_per_tf*1000:12.4f}      | {size_tf:8.2f}      | {gpu_mem_tf:10.2f}")
print(f"DLinear        | {train_time_dl:8.3f}      | {infer_total_dl:8.3f}      | {infer_per_dl*1000:12.4f}      | {size_dl:8.2f}      | {gpu_mem_dl:10.2f}")
print(f"Deep RVFL      | {train_time_rvfl:8.3f}    | {infer_total_rvfl:8.3f}    | {infer_per_rvfl*1000:12.4f}    | {size_rvfl:8.2f}    | {gpu_mem_rvfl:10.2f}")
print(f"CNN-LSTM       | {train_time_cnnlstm:8.3f} | {infer_total_cnnlstm:8.3f} | {infer_per_cnnlstm*1000:12.4f} | {size_cnnlstm:8.2f} | {gpu_mem_cnnlstm:10.2f}")
print(f"CNN-ANN        | {train_time_cnnann:8.3f}  | {infer_total_cnnann:8.3f}  | {infer_per_cnnann*1000:12.4f}  | {size_cnnann:8.2f}  | {gpu_mem_cnnann:10.2f}")
print(f"ARIMA-LSTM     | {train_time_arima_lstm:8.3f} | {infer_total_arima_lstm:8.3f} | {infer_per_arima_lstm*1000:12.4f} | {size_arima_lstm:8.2f} | {gpu_mem_arima_lstm:10.2f}")
print("---------------------------------------------------------------------------------\n")

## Statistical Table (Wilcoxon + Friedman)

In [ ]:

from scipy.stats import wilcoxon, friedmanchisquare
import pandas as pd
import numpy as np

# -------------------------------------------------------
# 1. Construct error dataframe
# -------------------------------------------------------
results_df = pd.DataFrame({
    "y_true"      : y_true_final,
    "ProphetBoost": pb_pred[-len(y_true_final):],
    "Transformer" : tf_pred[-len(y_true_final):],
    "DLinear"     : dl_pred[-len(y_true_final):],
    "RVFL"        : rvfl_pred[-len(y_true_final):],
    "CNN-LSTM"    : cnnlstm_pred[-len(y_true_final):],
    "CNN-ANN"     : cnnann_pred[-len(y_true_final):],
    "ARIMA-LSTM"  : arima_lstm_pred[-len(y_true_final):]
})

error_df = pd.DataFrame({
    model: abs(results_df["y_true"] - results_df[model])
    for model in results_df.columns if model != "y_true"
})

# -------------------------------------------------------
# 2. Wilcoxon Table Construction
# -------------------------------------------------------
baseline = "ProphetBoost"
rows = []

for model in error_df.columns:
    if model == baseline:
        continue

    W, p = wilcoxon(error_df[baseline], error_df[model])

    # Significance markers
    sig_025 = "a" if p < 0.025 else ""
    sig_05  = "a" if p < 0.05  else ""
    star_025 = "**" if p < 0.025 else "*"
    star_05  = "**" if p < 0.05 else "*"

    rows.append({
        "Compared Model": f"{baseline} vs {model}",
        "W (α=0.025)": f"{int(W)} {sig_025}",
        "p-value (α=0.025)": f"{p:.5f} {star_025}",
        "W (α=0.05)": f"{int(W)} {sig_05}",
        "p-value (α=0.05)": f"{p:.5f} {star_05}"
    })

wilcoxon_table = pd.DataFrame(rows)


# -------------------------------------------------------
# 3. Friedman Test
# -------------------------------------------------------
friedman_stat, friedman_p = friedmanchisquare(
    *[error_df[m] for m in error_df.columns]
)

friedman_result = {
    "Friedman Statistic": f"{friedman_stat:.4f}",
    "p-value": f"{friedman_p:.6f}",
    "Decision (α=0.05)": "Reject H₀" if friedman_p < 0.05 else "Fail to Reject H₀"
}

friedman_df = pd.DataFrame([friedman_result])


# -------------------------------------------------------
# 4. Print Results
# -------------------------------------------------------
print("\n======================= WILCOXON TABLE =======================")
print(wilcoxon_table.to_string(index=False))

print("\n======================= FRIEDMAN TEST =======================")
print(friedman_df.to_string(index=False))



## Without the feature selection

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import time
import pickle

import pandas as pd
import numpy as np
from prophet import Prophet

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error,
    mean_absolute_percentage_error, r2_score
)

import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import optuna
from optuna.samplers import TPESampler

import matplotlib.pyplot as plt

##############################################################################################################
# GPU / DEVICE SETUP
##############################################################################################################
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("\n✓ GPU Detected: Using CUDA")
    print("CUDA Version:", torch.version.cuda)
else:
    device = torch.device("cpu")
    print("\n× GPU NOT detected — using CPU")

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

##############################################################################################################
# UTILITIES FOR COMPLEXITY
##############################################################################################################
def get_model_size_mb(model, framework="torch"):
    """
    Save model to a temporary file and measure size in MB.
    framework: "torch", "xgb", "pickle"
    """
    if framework == "torch":
        tmp_file = "tmp_model.pth"
        torch.save(model.state_dict(), tmp_file)
    elif framework == "xgb":
        tmp_file = "tmp_model.json"
        model.save_model(tmp_file)
    else:  # generic pickle (Prophet, sklearn, statsmodels, etc.)
        tmp_file = "tmp_model.pkl"
        with open(tmp_file, "wb") as f:
            pickle.dump(model, f)

    size_mb = os.path.getsize(tmp_file) / (1024 * 1024)
    os.remove(tmp_file)
    return size_mb


def measure_inference_time(model, X, framework="torch"):
    """
    Measure total and per-sample inference time.
    Avoid tensor.numpy() to work around no-NumPy PyTorch builds.
    """
    if framework == "torch":
        model.eval()
        t0 = time.time()
        with torch.no_grad():
            _ = model(torch.tensor(X).float().to(device)).cpu()  # no .numpy()
        t1 = time.time()
    elif framework == "xgb":
        t0 = time.time()
        _ = model.predict(xgb.DMatrix(X.astype("float32")))
        t1 = time.time()
    elif framework == "sk":  # sklearn-like
        t0 = time.time()
        _ = model.predict(X)
        t1 = time.time()
    elif framework == "prophet":
        t0 = time.time()
        _ = model.predict(X)
        t1 = time.time()
    else:
        t0 = time.time()
        _ = model.predict(X)
        t1 = time.time()

    total = t1 - t0
    per_sample = total / len(X)
    return total, per_sample


def gpu_memory_used_mb():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        used = torch.cuda.max_memory_allocated() / (1024 * 1024)
        torch.cuda.reset_peak_memory_stats()
        return used
    else:
        return 0.0

##############################################################################################################
# STEP 1: Load + Preprocess
##############################################################################################################
df = pd.read_csv("totalload_new.csv", engine="python")
df = df.dropna(axis=1, how="all")

df["ds"] = pd.to_datetime(df["BASE_DT"], format="%d-%m-%y %H:%M", errors="coerce")
df = df.dropna(subset=["ds"]).sort_values("ds").reset_index(drop=True)

numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns
df[numeric_cols] = SimpleImputer(strategy="mean").fit_transform(df[numeric_cols])

df["y"] = df["TOTAL_LOAD"]

##############################################################################################################
# STEP 2: Prophet Decomposition (for features, not as a baseline)
##############################################################################################################
m = Prophet(weekly_seasonality=True, yearly_seasonality=True, daily_seasonality=True)
m.fit(df[["ds", "y"]])
fc = m.predict(df[["ds"]])

df["trend"] = fc["trend"].values
df["seasonality"] = fc["yearly"].values

##############################################################################################################
# STEP 3: Feature Engineering
##############################################################################################################
df["hour"] = df["ds"].dt.hour
df["dow"] = df["ds"].dt.dayofweek
df["weekend"] = (df["dow"] >= 5).astype(int)

df["lag1"] = df["y"].shift(1)
df["lag24"] = df["y"].shift(24)
df["roll3_mean"] = df["y"].rolling(3).mean()
df["roll3_std"] = df["y"].rolling(3).std()

weather_prefix = ["JEJU_", "GOSAN_", "SUNGSAN_", "SEOGWIPO_"]
weather_feats = [c for c in df.columns if any(c.startswith(p) for p in weather_prefix)]

for wf in weather_feats:
    df[f"{wf}_lag1"] = df[wf].shift(1)
    df[f"{wf}_lag24"] = df[wf].shift(24)

df = df.dropna().reset_index(drop=True)

base_feats = [
    "trend", "seasonality", "hour", "dow", "weekend",
    "lag1", "lag24", "roll3_mean", "roll3_std"
]
lag_weather = [c for c in df.columns if c.endswith("_lag1") or c.endswith("_lag24")]
full_feats = base_feats + weather_feats + lag_weather

##############################################################################################################
# STEP 4: Train/Test Split
##############################################################################################################
cut = pd.to_datetime("2019-01-01")
train = df[df["ds"] < cut].copy()
test  = df[df["ds"] >= cut].copy()

##############################################################################################################
# STEP 5: USE ALL FEATURES (NO SELECTION)
##############################################################################################################
selected_feats = full_feats
print(f"\nUsing all features without selection: {len(selected_feats)} features")

##############################################################################################################
# STEP 6: Scaling + Sequence Preparation
##############################################################################################################
scX = StandardScaler()
scY = StandardScaler()

Xtr = scX.fit_transform(train[selected_feats])
Xts = scX.transform(test[selected_feats])

ytr = scY.fit_transform(train["y"].values.reshape(-1, 1)).ravel()
yts = scY.transform(test["y"].values.reshape(-1, 1)).ravel()

def create_sequences(X, y, L):
    Xs, ys = [], []
    for i in range(len(X) - L):
        Xs.append(X[i:i + L])
        ys.append(y[i + L])
    return np.array(Xs), np.array(ys)

LOOKBACK = 24
Xtr_seq, ytr_seq = create_sequences(Xtr, ytr, LOOKBACK)
Xts_seq, yts_seq = create_sequences(Xts, yts, LOOKBACK)

##############################################################################################################
# MODEL 1 — ProphetBoost (XGBoost CPU hist)
##############################################################################################################
print("\n=== ProphetBoost (XGBoost, hist) ===")
dtrain = xgb.DMatrix(Xtr.astype("float32"), label=ytr.astype("float32"))
dtest  = xgb.DMatrix(Xts.astype("float32"), label=yts.astype("float32"))

xgb_params = {
    "objective": "reg:squarederror",
    "tree_method": "hist",      # CPU-safe
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "eval_metric": "mae"
}

cv = xgb.cv(
    xgb_params, dtrain, 1000,
    nfold=5, early_stopping_rounds=20,
    metrics="mae", as_pandas=True
)

best_rounds = len(cv)

t0 = time.time()
xgb_model = xgb.train(xgb_params, dtrain, best_rounds)
train_time_xgb = time.time() - t0

infer_total_xgb, infer_per_xgb = measure_inference_time(
    xgb_model, Xts, framework="xgb"
)
size_xgb = get_model_size_mb(xgb_model, framework="xgb")
gpu_mem_xgb = gpu_memory_used_mb()

pred_xgb_scaled = xgb_model.predict(dtest)
pred_xgb = scY.inverse_transform(pred_xgb_scaled.reshape(-1, 1)).ravel()

##############################################################################################################
# MODEL 2 — Transformer (VALID d_model % heads == 0)
##############################################################################################################
class TransformerModel(nn.Module):
    def __init__(self, d_model, num_heads, num_layers, dim_ff, input_dim):
        super().__init__()
        self.pe = self._pe(LOOKBACK, d_model)
        self.proj = nn.Linear(input_dim, d_model)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=num_heads,
            dim_feedforward=dim_ff, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, 1)

    def _pe(self, L, D):
        pos = torch.arange(L).unsqueeze(1)
        div = torch.exp(torch.arange(0, D, 2) * -(np.log(10000.0) / D))
        pe = torch.zeros(1, L, D)
        pe[0, :, 0::2] = torch.sin(pos * div)
        pe[0, :, 1::2] = torch.cos(pos * div)
        return pe

    def forward(self, x):
        x = self.proj(x) + self.pe[:, :x.size(1), :].to(device)
        x = self.encoder(x)
        return self.fc(x[:, -1, :])

##############################################################################################################
# Optuna Search Space (SAFE)
##############################################################################################################
def objective_tf(trial):

    d_model = trial.suggest_categorical("d_model", [32, 64, 128])
    num_heads = trial.suggest_categorical("num_heads", [1, 2, 4, 8])
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dim_ff = trial.suggest_categorical("dim_ff", [64, 128, 256])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch = trial.suggest_categorical("batch", [16, 32, 64])

    if d_model % num_heads != 0:
        return float("inf")

    model = TransformerModel(
        d_model, num_heads, num_layers, dim_ff,
        input_dim=Xtr_seq.shape[2]
    ).to(device)

    opt_ = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    loader = DataLoader(
        TensorDataset(
            torch.tensor(Xtr_seq).float(),
            torch.tensor(ytr_seq).float().unsqueeze(-1)
        ),
        batch_size=batch,
        shuffle=False   # avoid internal torch.randperm(...).numpy()
    )

    model.train()
    for _ in range(20):
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            opt_.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward()
            opt_.step()

    model.eval()
    with torch.no_grad():
        pred_t = model(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
        pred = np.array(pred_t.tolist())
        loss = mean_squared_error(yts_seq[:len(pred)], pred)

    return loss

print("\nSearching Transformer HPO...")
study_tf = optuna.create_study(direction="minimize")
study_tf.optimize(objective_tf, n_trials=10)
best_tf = study_tf.best_params
print("Transformer Best Params:", best_tf)

##############################################################################################################
# TRAIN FINAL TRANSFORMER
##############################################################################################################
model_tf = TransformerModel(
    d_model=best_tf["d_model"],
    num_heads=best_tf["num_heads"],
    num_layers=best_tf["num_layers"],
    dim_ff=best_tf["dim_ff"],
    input_dim=Xtr_seq.shape[2]
).to(device)

opt_tf = torch.optim.Adam(model_tf.parameters(), lr=best_tf["lr"])
loss_fn = nn.MSELoss()

loader_tf = DataLoader(
    TensorDataset(
        torch.tensor(Xtr_seq).float(),
        torch.tensor(ytr_seq).float().unsqueeze(-1)
    ),
    batch_size=best_tf["batch"],
    shuffle=False   # avoid RandomSampler here too
)

t0 = time.time()
model_tf.train()
for _ in range(50):
    for xb, yb in loader_tf:
        xb, yb = xb.to(device), yb.to(device)
        opt_tf.zero_grad()
        loss = loss_fn(model_tf(xb), yb)
        loss.backward()
        opt_tf.step()
train_time_tf = time.time() - t0

infer_total_tf, infer_per_tf = measure_inference_time(
    model_tf, Xts_seq, framework="torch"
)
size_tf = get_model_size_mb(model_tf, framework="torch")
gpu_mem_tf = gpu_memory_used_mb()

model_tf.eval()
with torch.no_grad():
    pred_tf_t = model_tf(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
    pred_tf_scaled = np.array(pred_tf_t.tolist())
pred_tf = scY.inverse_transform(pred_tf_scaled.reshape(-1, 1)).ravel()

##############################################################################################################
# MODEL 3 — DLINEAR
##############################################################################################################
class DLinear(nn.Module):
    def __init__(self, seq_len):
        super().__init__()
        self.fc = nn.Linear(seq_len, 1)

    def forward(self, x):
        x = x[:, :, 0]
        return self.fc(x)

def objective_dl(trial):

    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch = trial.suggest_categorical("batch", [16, 32, 64])

    model = DLinear(LOOKBACK).to(device)
    opti = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    loader = DataLoader(
        TensorDataset(
            torch.tensor(Xtr_seq).float(),
            torch.tensor(ytr_seq).float().unsqueeze(-1)
        ),
        batch_size=batch,
        shuffle=False
    )

    model.train()
    for _ in range(20):
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            opti.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward()
            opti.step()

    model.eval()
    with torch.no_grad():
        pred_t = model(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
        pred = np.array(pred_t.tolist())
        loss = mean_squared_error(yts_seq[:len(pred)], pred)

    return loss

print("\nSearching DLinear HPO...")
study_dl = optuna.create_study(direction="minimize")
study_dl.optimize(objective_dl, n_trials=10)
best_dl = study_dl.best_params
print("DLinear Best Params:", best_dl)

##############################################################################################################
# TRAIN FINAL DLINEAR
##############################################################################################################
model_dl = DLinear(LOOKBACK).to(device)
opt_dl = torch.optim.Adam(model_dl.parameters(), lr=best_dl["lr"])
loss_fn = nn.MSELoss()

loader_dl = DataLoader(
    TensorDataset(
        torch.tensor(Xtr_seq).float(),
        torch.tensor(ytr_seq).float().unsqueeze(-1)
    ),
    batch_size=best_dl["batch"],
    shuffle=False
)

t0 = time.time()
model_dl.train()
for _ in range(50):
    for xb, yb in loader_dl:
        xb, yb = xb.to(device), yb.to(device)
        opt_dl.zero_grad()
        loss = loss_fn(model_dl(xb), yb)
        loss.backward()
        opt_dl.step()
train_time_dl = time.time() - t0

infer_total_dl, infer_per_dl = measure_inference_time(
    model_dl, Xts_seq, framework="torch"
)
size_dl = get_model_size_mb(model_dl, framework="torch")
gpu_mem_dl = gpu_memory_used_mb()

model_dl.eval()
with torch.no_grad():
    pred_dl_t = model_dl(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
    pred_dl_scaled = np.array(pred_dl_t.tolist())
pred_dl = scY.inverse_transform(pred_dl_scaled.reshape(-1, 1)).ravel()

##############################################################################################################
# MODEL 4 — DEEP RVFL ENSEMBLE
##############################################################################################################
class DeepRVFL(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        return self.net(x)

print("\nTraining Deep RVFL Ensemble...")
rvfl_models = []
for _ in range(5):
    mdl = DeepRVFL(Xtr_seq.shape[1] * Xtr_seq.shape[2]).to(device)
    opti = torch.optim.Adam(mdl.parameters(), lr=1e-3)
    rvfl_models.append((mdl, opti))

loader_rvfl = DataLoader(
    TensorDataset(
        torch.tensor(Xtr_seq).float(),
        torch.tensor(ytr_seq).float().unsqueeze(-1)
    ),
    batch_size=32,
    shuffle=False
)

t0 = time.time()
for mdl, opti in rvfl_models:
    mdl.train()
    for _ in range(20):
        for xb, yb in loader_rvfl:
            xb, yb = xb.to(device), yb.to(device)
            opti.zero_grad()
            loss = loss_fn(mdl(xb), yb)
            loss.backward()
            opti.step()
train_time_rvfl = time.time() - t0

t0 = time.time()
preds_rvfl_scaled = []
for mdl, _ in rvfl_models:
    mdl.eval()
    with torch.no_grad():
        p_t = mdl(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
        p = np.array(p_t.tolist())
        preds_rvfl_scaled.append(p)
infer_total_rvfl = time.time() - t0
infer_per_rvfl = infer_total_rvfl / len(Xts_seq)

pred_rvfl_scaled = np.mean(preds_rvfl_scaled, axis=0)
pred_rvfl = scY.inverse_transform(pred_rvfl_scaled.reshape(-1, 1)).ravel()

size_rvfl = get_model_size_mb(rvfl_models[0][0], framework="torch")
gpu_mem_rvfl = gpu_memory_used_mb()

##############################################################################################################
# MODEL 5 — CNN-LSTM
##############################################################################################################
class CNNLSTM(nn.Module):
    """
    Conv1d over features (channels) + LSTM over time.
    """
    def __init__(self, input_dim, conv_channels=64, hidden_dim=128, num_layers=1):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim,
                               out_channels=conv_channels,
                               kernel_size=3,
                               padding=1)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(conv_channels, hidden_dim,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.relu(self.conv1(x))      # (B, C, T)
        x = x.permute(0, 2, 1)            # (B, T, C)
        out, _ = self.lstm(x)             # (B, T, H)
        return self.fc(out[:, -1, :])

print("\nTraining CNN-LSTM...")
model_cnnlstm = CNNLSTM(input_dim=Xtr_seq.shape[2]).to(device)
opt_cnnlstm = torch.optim.Adam(model_cnnlstm.parameters(), lr=1e-3)

loader_cnnlstm = DataLoader(
    TensorDataset(
        torch.tensor(Xtr_seq).float(),
        torch.tensor(ytr_seq).float().unsqueeze(-1)
    ),
    batch_size=64,
    shuffle=False
)

t0 = time.time()
model_cnnlstm.train()
for _ in range(50):
    for xb, yb in loader_cnnlstm:
        xb, yb = xb.to(device), yb.to(device)
        opt_cnnlstm.zero_grad()
        loss = loss_fn(model_cnnlstm(xb), yb)
        loss.backward()
        opt_cnnlstm.step()
train_time_cnnlstm = time.time() - t0

infer_total_cnnlstm, infer_per_cnnlstm = measure_inference_time(
    model_cnnlstm, Xts_seq, framework="torch"
)
size_cnnlstm = get_model_size_mb(model_cnnlstm, framework="torch")
gpu_mem_cnnlstm = gpu_memory_used_mb()

model_cnnlstm.eval()
with torch.no_grad():
    pred_cnnlstm_t = model_cnnlstm(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
    pred_cnnlstm_scaled = np.array(pred_cnnlstm_t.tolist())
pred_cnnlstm = scY.inverse_transform(pred_cnnlstm_scaled.reshape(-1, 1)).ravel()

##############################################################################################################
# MODEL 6 — CNN-ANN
##############################################################################################################
class CNNANN(nn.Module):
    """
    Conv1d over features + global pooling + MLP.
    """
    def __init__(self, input_dim, conv_channels=64):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim,
                               out_channels=conv_channels,
                               kernel_size=3,
                               padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.mlp = nn.Sequential(
            nn.Linear(conv_channels, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)           # (B, F, T)
        x = self.relu(self.conv1(x))     # (B, C, T)
        x = self.pool(x).squeeze(-1)     # (B, C)
        return self.mlp(x)               # (B, 1)

print("\nTraining CNN-ANN...")
model_cnnann = CNNANN(input_dim=Xtr_seq.shape[2]).to(device)
opt_cnnann = torch.optim.Adam(model_cnnann.parameters(), lr=1e-3)

loader_cnnann = DataLoader(
    TensorDataset(
        torch.tensor(Xtr_seq).float(),
        torch.tensor(ytr_seq).float().unsqueeze(-1)
    ),
    batch_size=64,
    shuffle=False
)

t0 = time.time()
model_cnnann.train()
for _ in range(50):
    for xb, yb in loader_cnnann:
        xb, yb = xb.to(device), yb.to(device)
        opt_cnnann.zero_grad()
        loss = loss_fn(model_cnnann(xb), yb)
        loss.backward()
        opt_cnnann.step()
train_time_cnnann = time.time() - t0

infer_total_cnnann, infer_per_cnnann = measure_inference_time(
    model_cnnann, Xts_seq, framework="torch"
)
size_cnnann = get_model_size_mb(model_cnnann, framework="torch")
gpu_mem_cnnann = gpu_memory_used_mb()

model_cnnann.eval()
with torch.no_grad():
    pred_cnnann_t = model_cnnann(torch.tensor(Xts_seq).float().to(device)).cpu().detach().view(-1)
    pred_cnnann_scaled = np.array(pred_cnnann_t.tolist())
pred_cnnann = scY.inverse_transform(pred_cnnann_scaled.reshape(-1, 1)).ravel()

##############################################################################################################
# MODEL 7 — ARIMA-LSTM (ARIMA + residual LSTM)
##############################################################################################################
print("\nTraining ARIMA-LSTM (ARIMA + residual LSTM)...")
y_train_raw = train["y"].values
y_test_raw  = test["y"].values

arima_order = (5, 1, 0)
arima_model = ARIMA(y_train_raw, order=arima_order)

t0 = time.time()
arima_result = arima_model.fit()
train_time_arima = time.time() - t0

arima_pred_train = arima_result.predict(start=0, end=len(y_train_raw)-1)
arima_pred_test  = arima_result.predict(start=len(y_train_raw),
                                        end=len(y_train_raw)+len(y_test_raw)-1)

residual_train = y_train_raw - arima_pred_train
scRes = StandardScaler()
res_tr_scaled = scRes.fit_transform(residual_train.reshape(-1, 1)).ravel()

def create_sequences_res(X, y, L):
    Xs, ys = [], []
    for i in range(len(X) - L):
        Xs.append(X[i:i + L])
        ys.append(y[i + L])
    return np.array(Xs), np.array(ys)

Xtr_res_seq, ytr_res_seq = create_sequences_res(Xtr, res_tr_scaled, LOOKBACK)

class ResidualLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

print("\nTraining residual LSTM...")
model_res_lstm = ResidualLSTM(input_dim=Xtr_res_seq.shape[2]).to(device)
opt_res = torch.optim.Adam(model_res_lstm.parameters(), lr=1e-3)

loader_res = DataLoader(
    TensorDataset(
        torch.tensor(Xtr_res_seq).float(),
        torch.tensor(ytr_res_seq).float().unsqueeze(-1)
    ),
    batch_size=64,
    shuffle=False
)

t0 = time.time()
model_res_lstm.train()
for _ in range(50):
    for xb, yb in loader_res:
        xb, yb = xb.to(device), yb.to(device)
        opt_res.zero_grad()
        loss = loss_fn(model_res_lstm(xb), yb)
        loss.backward()
        opt_res.step()
train_time_res_lstm = time.time() - t0
train_time_arima_lstm = train_time_arima + train_time_res_lstm

# Inference: ARIMA test + LSTM residual
t0 = time.time()
arima_test_seq = arima_pred_test[LOOKBACK:]
with torch.no_grad():
    res_pred_t = model_res_lstm(
        torch.tensor(Xts_seq).float().to(device)
    ).cpu().detach().view(-1)
    res_pred_scaled = np.array(res_pred_t.tolist())

res_pred = scRes.inverse_transform(res_pred_scaled.reshape(-1, 1)).ravel()
pred_arima_lstm = arima_test_seq + res_pred
infer_total_arima_lstm = time.time() - t0
infer_per_arima_lstm = infer_total_arima_lstm / len(Xts_seq)

size_arima = get_model_size_mb(arima_result, framework="pickle")
size_res_lstm = get_model_size_mb(model_res_lstm, framework="torch")
size_arima_lstm = size_arima + size_res_lstm
gpu_mem_arima_lstm = gpu_memory_used_mb()

##############################################################################################################
# FINAL METRICS 
##############################################################################################################
def metrics(ytrue, ypred):
    mae = mean_absolute_error(ytrue, ypred)
    mse = mean_squared_error(ytrue, ypred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(ytrue, ypred) * 100
    r2 = r2_score(ytrue, ypred)
    return mae, mse, rmse, mape, r2

# Align all sequence-based models to same horizon (RVFL length)
y_true_final = test["y"].values[-len(pred_rvfl):]

pb_pred         = pred_xgb[-len(y_true_final):]
tf_pred         = pred_tf[-len(y_true_final):]
dl_pred         = pred_dl[-len(y_true_final):]
rvfl_pred       = pred_rvfl
cnnlstm_pred    = pred_cnnlstm[-len(y_true_final):]
cnnann_pred     = pred_cnnann[-len(y_true_final):]
arima_lstm_pred = pred_arima_lstm[-len(y_true_final):]



In [ ]:
print("\n================== FINAL RESULTS ==================")
print("Model              MAE        MSE        RMSE       MAPE      R2")
print("--------------------------------------------------------------------------")
print(f"ProphetBoost   {metrics(y_true_final, pb_pred)}")
print(f"Transformer    {metrics(y_true_final, tf_pred)}")
print(f"DLinear        {metrics(y_true_final, dl_pred)}")
print(f"Deep RVFL      {metrics(y_true_final, rvfl_pred)}")
print(f"CNN-LSTM       {metrics(y_true_final, cnnlstm_pred)}")
print(f"CNN-ANN        {metrics(y_true_final, cnnann_pred)}")
print(f"ARIMA-LSTM     {metrics(y_true_final, arima_lstm_pred)}")
print("==========================================================================\n")

##############################################################################################################
# MODEL COMPLEXITY SUMMARY
##############################################################################################################
print("\n================ MODEL COMPLEXITY ==================")
print("Model          | Train(s) | Infer(s) | Infer/sample(ms) | Size(MB) | GPU Mem(MB)")
print("---------------------------------------------------------------------------------")
print(f"ProphetBoost   | {train_time_xgb:8.3f} | {infer_total_xgb:8.3f} | {infer_per_xgb*1000:12.4f} | {size_xgb:8.2f} | {gpu_mem_xgb:10.2f}")
print(f"Transformer    | {train_time_tf:8.3f} | {infer_total_tf:8.3f} | {infer_per_tf*1000:12.4f} | {size_tf:8.2f} | {gpu_mem_tf:10.2f}")
print(f"DLinear        | {train_time_dl:8.3f} | {infer_total_dl:8.3f} | {infer_per_dl*1000:12.4f} | {size_dl:8.2f} | {gpu_mem_dl:10.2f}")
print(f"Deep RVFL      | {train_time_rvfl:8.3f} | {infer_total_rvfl:8.3f} | {infer_per_rvfl*1000:12.4f} | {size_rvfl:8.2f} | {gpu_mem_rvfl:10.2f}")
print(f"CNN-LSTM       | {train_time_cnnlstm:8.3f} | {infer_total_cnnlstm:8.3f} | {infer_per_cnnlstm*1000:12.4f} | {size_cnnlstm:8.2f} | {gpu_mem_cnnlstm:10.2f}")
print(f"CNN-ANN        | {train_time_cnnann:8.3f} | {infer_total_cnnann:8.3f} | {infer_per_cnnann*1000:12.4f} | {size_cnnann:8.2f} | {gpu_mem_cnnann:10.2f}")
print(f"ARIMA-LSTM     | {train_time_arima_lstm:8.3f} | {infer_total_arima_lstm:8.3f} | {infer_per_arima_lstm*1000:12.4f} | {size_arima_lstm:8.2f} | {gpu_mem_arima_lstm:10.2f}")
print("---------------------------------------------------------------------------------\n")
